<a href="https://colab.research.google.com/github/elizagb/Google-Earth-Engine-Practice/blob/main/Correct_Unsupervised_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsupervised classification of Holiday Farm Fire region using Sentinel 2 data:

### Set up the environment:

In [19]:

# earth engine and statistical analysis
from IPython.display import Image
import ee, datetime
import geemap.core as geemap
import pandas as pd
from pylab import *
import seaborn as sns

# file management
import os
import glob

# geospatial and plotting modules
# import geopandas as gpd

import geemap.foliumap as geemap
import folium
import matplotlib.pyplot as plt
%matplotlib inline

import altair as alt


### Authenticate earth engine:


In [20]:
# Trigger the authentication flow.
ee.Authenticate()

True

In [21]:
# start up the session
ee.Initialize(project='ee-egblack')

### Sentinel-2 indices:

In [60]:
# Normalized burn ratio
def senNBR(image):
  return image.normalizedDifference(['B8', 'B12'])

# NDVI for Sentinel-2 bands
def senNDVI(image):
    return image.normalizedDifference(['B8', 'B4'])

### Create target area for analysis:

In [51]:
# Define the Holiday Farm Fire center
center = ee.Geometry.Point([-122.418960, 44.163945])

# Define a region around the center (e.g., a 20 km buffer)
region = center.buffer(20000)  # 20 km radius

### Sentinel-2 image collection:

In [72]:
# Load Sentinel-2 surface reflectance imagery for the post-fire period
image = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(region)
    .filterDate("2024-03-01", "2024-09-30")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))  # Filter low-cloud images
    .median()  # Take median composite to reduce noise
    .clip(region)
)

In [69]:
# Print number of bands
print(image.bandNames().length().getInfo())

# Print band names
print(image.bandNames().getInfo())

26
['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE']


### Add NBR and NDVI bands:


In [75]:
# Compute NDVI and add it as a new band
ndvi = senNDVI(image)

if 'NDVI' not in image.bandNames().getInfo():
    image = image.addBands(senNDVI(image).rename('NDVI'))

# Print band names to confirm NDVI is added
print(image.bandNames().getInfo())


['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE', 'NDVI']


In [77]:
# Compute NBR and add it as a new band
nbr = senNBR(image)

if 'NBR' not in image.bandNames().getInfo():
    image = image.addBands(senNBR(image).rename('NBR'))

# Print band names to confirm NDVI is added
print(image.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE', 'NDVI', 'NBR']


In [79]:
# NOTE: this unsupervised classification doesn't work if you apply it directly to input
# Need to apply clustering to the input filtered by bands

# select the 10m resolution bands to use in the classification
# see what each band means here: https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/bands/

# blue, green, red, NIR
# band_clusters = ['B2', 'B3', 'B4', 'B8']
band_clusters = ['B2', 'B3', 'B4', 'B8', 'NDVI', 'NBR']

filtered_image = image.select(band_clusters)

In [80]:
# Display the sample region
m = geemap.Map()
m.set_center(-122.418960, 44.163945, 9)  # Zoom in on fire area
m.add_layer(ee.Image().paint(region, 0, 2), {}, "region")

In [81]:
# Make the training dataset
training = filtered_image.sample(region=region, scale=10, numPixels=5000)  # Sentinel-2 has 10m resolution

# Instantiate the clusterer and train it
clusterer = ee.Clusterer.wekaKMeans(10).train(training)

# Cluster the input using the trained clusterer
result = filtered_image.cluster(clusterer)

In [82]:
# Display the clusters with random colors
m.add_layer(result.randomVisualizer(), {}, "clusters")

# Show the map
m